In [2]:
# Agregar la carpeta raíz del proyecto al path
import os
import sys
project_root = os.path.abspath("..")
sys.path.append(project_root)

In [7]:
print(project_root)

d:\PruebasDS\CrediBanco


In [5]:
import pandas as pd
import msoffcrypto
import pandas as pd
import io
import os
from dotenv import load_dotenv

In [11]:
# Cargar variables del archivo .env
load_dotenv()
contrasena = os.getenv("EXCEL_PASSWORD")

# Ruta al archivo Excel protegido
ruta_archivo = os.path.join(project_root,'data','Transacciones_Prueba_Especialista_Prevencion_Fraude.xlsx')
print(ruta_archivo)

# Desencriptar el archivo
with open(ruta_archivo, 'rb') as f:
    archivo_encriptado = msoffcrypto.OfficeFile(f)
    archivo_encriptado.load_key(password=contrasena)
    
    archivo_desencriptado = io.BytesIO()
    archivo_encriptado.decrypt(archivo_desencriptado)

# Leer las hojas
df_transacciones = pd.read_excel(archivo_desencriptado, sheet_name='Transacciones', engine='openpyxl')
archivo_desencriptado.seek(0)
df_diccionario = pd.read_excel(archivo_desencriptado, sheet_name='Diccionario', engine='openpyxl')

# Mostrar datos
print(df_transacciones.head())
print(df_diccionario.head())


d:\PruebasDS\CrediBanco\data\Transacciones_Prueba_Especialista_Prevencion_Fraude.xlsx
         UTRNNO            HPAN            FECHA_TRX            HORA_TRX  \
0  145736384287  93549332211627  2024-07-01 00:00:00 2024-07-01 08:12:00   
1  145736384281  84409332211536  2024-07-01 00:00:00 2024-07-01 18:47:00   
2  145736384252  84400332211005  2024-07-01 00:00:00 2024-07-01 02:05:00   
3  145736384246  93526332211167  2024-07-01 00:00:00 2024-07-01 13:51:00   
4  145736384197  93530332211347  2024-07-01 00:00:00 2024-07-01 15:47:00   

  COUNTRY_CODE  TERM_TYPE    AMOUNT_1 CUR_CODE BIN_CURRENCY     MCC  ...  \
0           CO        8.0  19429000.0      170          170  4511.0  ...   
1           CO        8.0   2050000.0      170          170  5818.0  ...   
2           CO        2.0       100.0      170          170  9399.0  ...   
3           CO        2.0  15290000.0      170          170  5813.0  ...   
4           CO        2.0  31201500.0      170          170  5541.0  ...   

